In [ ]:
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import numpy as np
import esm
import torch
import os


def esm_embed(sequence_list):
    model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
    batch_converter = alphabet.get_batch_converter()
    model.eval()
    
    peptide_sequence_list = []
    for seq in sequence_list:
        format_seq = [seq, seq]
        tuple_sequence = tuple(format_seq)
        peptide_sequence_list.append(tuple_sequence)
    
    data = peptide_sequence_list
    batch_labels, batch_strs, batch_tokens = batch_converter(data)
    batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
    
    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[6], return_contacts=True)
    token_representations = results["representations"][6]
    
    sequence_representations = []
    for i, token_len in enumerate(batch_lens):
        each_seq_rep = token_representations[i, 1:token_len - 1].mean(0).tolist()  
        sequence_representations.append(each_seq_rep)

    return pd.DataFrame(sequence_representations)


def open_file():
    file_path = filedialog.askopenfilename(filetypes=[("Excel Files", "*.xlsx")])
    file_path_entry.delete(0, tk.END)
    file_path_entry.insert(tk.END, file_path)


def process_file():
    file_path = file_path_entry.get()
    output_file_path = os.path.splitext(file_path)[0] + "_embedding.xlsx"
    
    df = pd.read_excel(file_path)
    sequences = df["Sequence"].tolist()
    
    embedding_df = esm_embed(sequences)
    
    embedding_df.to_excel(output_file_path, index=False)
    
    result_text.delete(1.0, tk.END)
    result_text.insert(tk.END, f"The embedding result has been saved to {output_file_path}")


def process_input_sequence():
    input_sequence = input_sequence_entry.get()
    
    embedding_df = esm_embed([input_sequence])
    
    embedding_df.to_excel("embedding_result.xlsx", index=False)
    
    result_text.delete(1.0, tk.END)
    result_text.insert(tk.END, "The embedding result has been saved to embedding_result.xlsx")


window = tk.Tk()

# 选择文件部分
file_path_label = tk.Label(window, text="select file:")
file_path_label.pack()
file_path_entry = tk.Entry(window)
file_path_entry.pack()
file_path_button = tk.Button(window, text="select file", command=open_file)
file_path_button.pack()

# 处理文件部分
process_file_button = tk.Button(window, text="process file", command=process_file)
process_file_button.pack()

# 输入序列部分
input_sequence_label = tk.Label(window, text="input sequence:")
input_sequence_label.pack()
input_sequence_entry = tk.Entry(window)
input_sequence_entry.pack()
process_sequence_button = tk.Button(window, text="process sequence", command=process_input_sequence)
process_sequence_button.pack()

# 结果显示部分
result_label = tk.Label(window, text="result:")
result_label.pack()
result_text = tk.Text(window)
result_text.pack()

window.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\py\anaconda3\envs\sdsd_torch\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\RH\AppData\Local\Temp\ipykernel_10184\4268546074.py", line 47, in process_file
    df = pd.read_excel(file_path)
  File "D:\py\anaconda3\envs\sdsd_torch\lib\site-packages\pandas\util\_decorators.py", line 211, in wrapper
    return func(*args, **kwargs)
  File "D:\py\anaconda3\envs\sdsd_torch\lib\site-packages\pandas\util\_decorators.py", line 331, in wrapper
    return func(*args, **kwargs)
  File "D:\py\anaconda3\envs\sdsd_torch\lib\site-packages\pandas\io\excel\_base.py", line 482, in read_excel
    io = ExcelFile(io, storage_options=storage_options, engine=engine)
  File "D:\py\anaconda3\envs\sdsd_torch\lib\site-packages\pandas\io\excel\_base.py", line 1652, in __init__
    ext = inspect_excel_format(
  File "D:\py\anaconda3\envs\sdsd_torch\lib\site-packages\pandas\io\excel\_base.py", l